In [21]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [22]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("Spark Example").getOrCreate()

In [23]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# This is useful if direct URL reading continues to fail
spark.sparkContext.addFile(url)

local_path = SparkFiles.get("home_sales_revised.csv")

df = spark.read.csv(local_path, header=True, inferSchema=True)

In [24]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

In [25]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [26]:
avg_price_per_year = spark.sql("""
    SELECT YEAR(date) AS Sale_Year, ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY Sale_Year
    ORDER BY Sale_Year
""")

In [29]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

# Install Findspark
!pip install -q findspark

import os

# Set environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# Import findspark and initialize
import findspark
findspark.init()

# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("Spark Example").getOrCreate()

# Read in the AWS S3 bucket into a DataFrame
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark context
spark.sparkContext.addFile(url)
local_path = SparkFiles.get("home_sales_revised.csv")

# Read the CSV file
df = spark.read.csv(local_path, header=True, inferSchema=True)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("home_sales")

# Query to get the average price for a four-bedroom house sold per year, rounded to two decimal places
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_per_year = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(avg_price_per_year).show()


The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.3.1-bin-hadoop3.tgz'


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



All Code above came from ASKbcs, tutor helped me go through each step.

In [35]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_per_year_built =  """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(avg_price_per_year_built).show()



+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [38]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_specific_criteria = """
SELECT
  date_built AS Year_Built,
  ROUND(AVG(price), 2) AS Avg_Price
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(avg_price_specific_criteria).show()

+----------+---------+
|Year_Built|Avg_Price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [41]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Measure execution time
start_time = time.time()

# Updated SQL query
avg_price_per_view_rating = spark.sql("""
    SELECT
      view AS View,
      ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

# Show results
avg_price_per_view_rating.show()

# Stop the Spark session
spark.stop()

# Measure runtime
end_time = time.time()
runtime = end_time - start_time
print(f"--- Query 6 runtime: {runtime} seconds ---")

+----+----------+
|View| Avg_Price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Query 6 runtime: 1.7403883934020996 seconds ---


In [43]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

Py4JJavaError: An error occurred while calling o165.cacheTable.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConnection.run(ClientServerConnection.java:106)
java

In [44]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached? {is_cached}")

Is home_sales cached? False


In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
start_time_cached = time.time()

avg_price_per_view_rating_cached = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS Avg_Price
    FROM home_sales
    GROUP BY View
    HAVING AVG(Price) >= 350000
    ORDER BY View DESC
""")
avg_price_per_view_rating_cached.show()

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print(f"--- Query 9 (cached) runtime: {runtime_cached} seconds ---")


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("path_to_save_formatted_parquet")

In [ ]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("path_to_save_formatted_parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
start_time_parquet = time.time()

avg_price_per_view_rating_parquet = spark.sql("""
    SELECT View, ROUND(AVG(Price), 2) AS Avg_Price
    FROM parquet_home_sales
    GROUP BY View
    HAVING AVG(Price) >= 350000
    ORDER BY View DESC
""")
avg_price_per_view_rating_parquet.show()

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet
print(f"--- Query 13 (parquet) runtime: {runtime_parquet} seconds ---")

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached_after_uncache = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached after uncache? {is_cached_after_uncache}")
